# Opprett part
Denne notebooken beskriver hvordan en går frem for å gjøre klar en part til test for skattemeldingen etter den er funnet med [TENOR testdatasøk](https://www.skatteetaten.no/skjema/testdata/)


In [1]:
from docs.test.testinnsending.skatteetaten_api import get_access_token
from requests import post

token = get_access_token()

/Users/martin.unhjem/work/skattemeldingen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


The token is good, expires at 11:37:14 (3600 seconds from now)


127.0.0.1 - - [22/Nov/2024 10:37:14] "GET /token?code=yG9Ad9DGsz-AbdbXDo6auw.OBGN6es5rpoDs64B8wD4kw&iss=https%3A%2F%2Ftest.idporten.no&state=mcNmlTyku97ELTOW3gg1Gg HTTP/1.1" 200 -


In [2]:
inntektsår = "2024"
identifikator = "27867196108"
url_path = f"/api/skattemelding/v2/utsattfristsoeknad/{identifikator}"
url = "http://ekstern-token-validering.fastsetting.formueinntekt.skattemelding.sit.apps.utv10.paas.skead.no"
base_url = "https://idporten-api-sbstest.sits.no"

In [3]:
klienter = {
    "referanse": "fra ekstern",
    "klienter": [
        {
            "identifikator": {
                "foedselsnummer": "27867196108"
            },
            "naering": False,
            "navn": "Per Tyrker",
            "epost": "per@tyrker.no"
        },
        {
            "identifikator": {
                "organisasjonsnummer": "310783145"
            },
            "naering": True,
            "navn": "kari Nordmann",
            "epost": "kari@Nordmann.no"
        }
    ]
}

In [4]:
utsatt_frist_request = post(base_url+url_path, headers=token, json=klienter)

In [5]:
utsatt_frist_request.request.body

b'{"referanse": "fra ekstern", "klienter": [{"identifikator": {"foedselsnummer": "27867196108"}, "naering": false, "navn": "Per Tyrker", "epost": "per@tyrker.no"}, {"identifikator": {"organisasjonsnummer": "310783145"}, "naering": true, "navn": "kari Nordmann", "epost": "kari@Nordmann.no"}]}'

In [6]:
utsatt_frist_request.text

'{"type":"about:blank","title":"Bad Request","status":400,"detail":"Failed to read request","instance":"/api/skattemelding/v2/utsattfristsoeknad/27867196108"}'